In [ ]:
import numpy as np

In [ ]:
images_url = "./CUB_200_2011/images.txt"
train_test_split_url = "./CUB_200_2011/train_test_split.txt"
classes_url = "./CUB_200_2011/classes.txt"
image_class_labels_url = "./CUB_200_2011/image_class_labels.txt"

Loading .txt file

In [ ]:
image = np.genfromtxt(images_url , delimiter=' ', dtype=str) #<image_id> <image_name>

train_test_split = np.genfromtxt(train_test_split_url, delimiter=" ", dtype=str) #<image_id> <is_training_image>

classes = np.genfromtxt(classes_url, delimiter=" ", dtype=str) #<class_id> <class_name>

image_class_labels = np.genfromtxt(image_class_labels_url, delimiter=" ", dtype=str).astype(int) #<image_id> <class_id>

<table>
    <tr>
        <td>image</td>
        <td>&lt;image_id&gt; &lt;image_name&gt;</td>
    </tr>
    <tr>
        <td>train_test_split</td>
        <td>&lt;image_id&gt; &lt;is_training_image&gt;</td>
    </tr>
    <tr>
        <td>classes</td>
        <td>&lt;class_id&gt; &lt;class_name&gt;</td>
    </tr>
    <tr>
        <td>image_class_labels</td>
        <td>&lt;image_id&gt; &lt;class_id&gt;</td>
    </tr>
</table>

In [ ]:
image = {int(row[0]): "CUB_200_2011/images/"+row[1] for row in image}

train_test_split = {int(row[0]): int(row[1]) for row in train_test_split}

classes = {int(row[0]): row[1] for row in classes}

image_class_labels = {int(row[0]): int(row[1]) for row in image_class_labels}

In [ ]:
row = 5
print(row, image[row])
print(row, train_test_split[row])
print(row, classes[row])
print(row, image_class_labels[row])

In [ ]:
# image_train = {key: value for key, value in  train_test_split.items() if value == 1}
# image_test = {key: value for key, value in  train_test_split.items() if value == 0}
image_train = np.array([key for key, value in train_test_split.items() if value == 1])
image_test = np.array([key for key, value in train_test_split.items() if value == 0])

image_split = {
    "train": image_train,
    "test": image_test
}

n_train = len(image_train)
n_test = len(image_test)
n_classes = len(classes)

print("Number of Images:", len(image))
print(f"n_train: {n_train}")
print(f"n_test: {n_test}")
print(f"n_classes: {n_classes}")

<table>
    <tr>
        <td>image_train</td>
        <td>list of &lt;image_id&gt;</td>
        <td>&lt;is_training_image&gt; == 1</td>
    </tr>
    <tr>
        <td>image_test</td>
        <td>list of &lt;image_id&gt;</td>
        <td>&lt;is_training_image&gt; == 0</td>
    </tr>
</table>

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def showImage(image):
    plt.imshow(plt.imread(image))
    plt.axis(False)
    plt.show()

In [ ]:
# showImage(image[10])

## Model
### EfficientNet_B1_V2

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

In [ ]:
# hyper parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_out_ftrs = 200 # number of classes for out classification is 200
image_input_size = 244 # 244x244 image; imagas are resized to this size
batch_size = 4
# batch_size_test = 1
learning_rate = 0.001
num_epoch = 20

In [ ]:
model = models.efficientnet_b1(weights="DEFAULT")

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters in original model: {total_params}")

num_params = sum(param.numel() for param in model.parameters() if param.requires_grad)
print(f"Total number of training parameters in original model: {num_params}")

In [ ]:
# printing the model
model

In [ ]:
# freeze all layers for fine tuning (not doing this takes it very long to train)
for param in model.parameters():
    param.requires_grad = False

# number of inputs in last layer
num_ftrs = model.classifier[1].in_features

model.classifier[1] = nn.Linear(num_ftrs, num_out_ftrs)

# printing the last layer : classifier
print(model.classifier)

model = model.to(device)

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters in fine-tuned model: {total_params}")

num_params = sum(param.numel() for param in model.parameters() if param.requires_grad)
print(f"Total number of training parameters in fine-tuned model: {num_params}")

### Loading dataset in torch dataloader

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import save_image
import torchvision.transforms as transforms
from PIL import Image
import time
import os
import copy
import sys

In [ ]:
class ExpandGrayscales:
    def __call__(self, sample):
        return sample.expand(3,-1,-1)

transform = transforms.Compose(
    [
        transforms.Resize((image_input_size,image_input_size)),
        transforms.ToTensor(),
        ExpandGrayscales()
    ]
)

augmentation_trasform = transforms.Compose([
    transforms.Resize((image_input_size,image_input_size)),
    transforms.ColorJitter(
        brightness=0.5,
        contrast=0.2,
        saturation=0.2,
        hue=0.05
    ),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=30),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    ExpandGrayscales()
])

In [ ]:
class CUBDataset(Dataset):
    def __init__(self, image_id, id_to_url, image_class_labels, transform):
        self.x = [id_to_url[x] for x in image_id]
        self.y = [image_class_labels[x] for x in image_id]
        self.n_samples = len(image_id)
        self.transform = transform
    
    def __getitem__(self, index):
        img = Image.open(self.x[index])
        img = self.transform(img)
        return img, self.y[index]-1
    
    def __len__(self):
        return self.n_samples

### Data Augmentation Process

In [ ]:
augment_train_dataset = CUBDataset(
    image_id=image_split["train"],
    id_to_url=image,
    image_class_labels=image_class_labels,
    transform=augmentation_trasform
)

augment_batch_size = 20

augment_train_dataloader = DataLoader(
    dataset=augment_train_dataset,
    batch_size=augment_batch_size,
    shuffle=False
)

In [ ]:
img_num = int(time.time())
folder_path = "CUB_200_2011/aug_images"

if not os.path.exists(folder_path):

    os.makedirs(folder_path)
    for _ in range(3):
        for images, labels in augment_train_dataloader:
            for i in range(images.shape[0]):
                file_path = folder_path + "/" + str(int(labels[i])+1) + "_img" + str(img_num) + ".png"
                save_image(images[i], file_path)
                image[img_num] = file_path
                image_split["train"] = np.append(image_split["train"], img_num)
                image_class_labels[img_num] = (int(labels[i])+1)
                img_num += 1

else:
    file_names = os.listdir(folder_path)
    for file_name in file_names:
        class_id = int(file_name.split("_")[0])
        file_path = folder_path + "/" + file_name
        image[img_num] = file_path
        image_split["train"] = np.append(image_split["train"], img_num)
        image_class_labels[img_num] = (class_id)
        img_num += 1

In [ ]:
np.random.seed(42)
np.random.shuffle(image_split["train"])
np.random.shuffle(image_split["test"])

In [ ]:
# print(len(image), len(image_split["train"]), len(image_split["test"]), len(image_class_labels))
print(f"Train + Test: {len(image)}")
print(f"Train: {len(image_split["train"])}")
print(f"Test: {len(image_split["test"])}")
print(f"Image Class Label Length: {len(image_class_labels)}")

### Creating Actual Train / Test Datasets

In [ ]:
datasets = {
    x: CUBDataset(
        image_id=image_split[x],
        id_to_url=image, 
        image_class_labels=image_class_labels,
        transform=transform
    )
    for x in ["train", "test"]
}

In [ ]:
dataloaders = {
    x: DataLoader(
        dataset=datasets[x],
        batch_size=batch_size,
        shuffle=False
    )
    for x in ["train", "test"]
}

In [ ]:
dataset_sizes = {
    x: len(datasets[x])
    for x in ["train", "test"]
}
dataset_sizes

In [ ]:
dataloader_sizes = {
    x: len(dataloaders[x])
    for x in ["train", "test"]
}
dataloader_sizes

In [ ]:
# dataiter = iter(dataloader_train)
# j = 1
# for i in dataloader_train:
#     features, labels = next(dataiter)
#     # if features.shape != torch.Size([1,3,244,244]):
#     features = features.expand(-1,3,244,244).numpy()
#     print(features.shape)
#     plt.imshow(features.squeeze().transpose(1,2,0))
#     plt.show()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    params=model.parameters(),
    lr=learning_rate
)

In [ ]:
# not required in Adam optimizer
step_lr_schedular = torch.optim.lr_scheduler.StepLR(
    optimizer=optimizer,
    step_size=7,
    gamma=0.1
)

In [ ]:
def train_model(model, criterion, optimizer, schedular=None, num_epoch=20, save_checkpoint=False):
    print(f"Training Started on {device}")
    train_time = 0

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epoch):
        print(f"Epoch {epoch+1}/{num_epoch}", end="")

        #each epoch has a training and a validation phase
        for phase in ["train", "test"]:
            time_start = time.time()

            model.train() if phase == "train" else model.eval()

            running_loss = 0.0
            running_corrects = 0.0

            for i, (images, labels) in enumerate(dataloaders[phase]):
                images = images.to(device)
                labels = labels.to(device)
                
                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(images)
                    output_one_hot = torch.argmax(outputs, dim=1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training loop
                    if phase == "train":
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                if phase == "train" and (i+1) % int(dataloader_sizes[phase]/10) == 0:
                    print("-", end=" ")
                        
                running_loss += loss.item() * images.size(0)
                running_corrects += torch.sum(output_one_hot == labels.data)

            if schedular != None and phase == "train":
                    schedular.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = 100 * running_corrects.double() / dataset_sizes[phase]
            
            epoch_time = time.time() - time_start
            if phase == "train":
                train_time += epoch_time

            print(f"\n{phase} Loss: {epoch_loss:.2f} Acc: {epoch_acc:.2f}%", f"Time_Taken: {epoch_time//60:.0f}m {epoch_time%60:.0f}s", end="")

            # deep copy the best model
            if phase == "train" and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print("\n")

    print(f"Training Finished in {train_time//60:.0f}m {train_time%60:.0f}s")
    print(f"Best Test Accuracy: {best_acc:4f}")
    print(model.__class__.__name__ + "_checkpoint_best_acc_" + str(f"{best_acc:.4f}") + "_" + "_epoch_" + str(num_epoch) + "_optim_" + optimizer.__class__.__name__ + "_criterion_" + criterion.__class__.__name__)

    model.load_state_dict(best_model_wts)

    if save_checkpoint == True:
        checkpoint = {
            "epoch": num_epoch,
            "criterion": criterion,
            "model_state": model.state_dict(),
            "optim_state": optimizer.state_dict()
        }
        file_name = model.__class__.__name__ + "_checkpoint_best_acc_" + str(f"{best_acc:.4f}") + "_" + "_epoch_" + str(num_epoch) + "_optim_" + optimizer.__class__.__name__ + "_criterion_" + criterion.__class__.__name__ + ".pth"
        torch.save(checkpoint, file_name)
    
    return model

In [ ]:
model = train_model(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    num_epoch=num_epoch,
    save_checkpoint=False
)